In [5]:
import json
import uuid
import numpy as np
from tqdm import tqdm
from planckmc.config import CONFIG

In [6]:
#The warning on Response_Update stands here as well; this code is to get started with detector building, but many features are still missing
#(particularly in the other two updates)

Version = 3 # This should be the same as the version in config.ini

# Make sure these three are even
Sensors_X = 4
Sensors_Y = 4
Sensors_Z = 4

# These Three Determine the Detector Structure
Dist_btw_Sen_X = 2
Dist_btw_Sen_Y = 2
Dist_btw_Sen_Z = 2

# NOTE: The sensor positions will be on multiples of the two value sets above PLUS 1. So, for example, with Sensors_X = 4 and Dist_btw_Sen_X = 2,
# the first sensor will be placed at (-(4/2) * 2) + 1 = -3. The subsequent positions of this setup would be -1, 1, and 3, giving the 4 sensors

# Make sure to update config.ini with the correct sphere radius (The edge of the detector (e.g. in X) is given by: mag((-Sesnors_X / 2) * Dist_btw_Sen_X) + 1) or (((Sensors_X / 2) - 1) * Dist_btw_Sen_X) + 1

# The serial number is set to 6 unique alphanumeric digits, covering ~2.5 billions possible sensor combinations

# There are also data inputs in Response_Update

In [7]:
_VERSION = int(CONFIG['Detector Config']['Version'])

if Version != _VERSION:
    raise ValueError("You are running the incorrect version of the configuration file.\nExiting.")

if Sensors_X % 2 != 0 or Sensors_Y % 2 != 0 or Sensors_Z % 2 != 0:
    raise ValueError("Please input an even number of sensors in each dimension")


dictionary = {"version": Version, "sensors": 0, "detectors": []}
json_object = json.dumps(dictionary)
with open("position.json", "w") as outfile: 
    outfile.write(json_object)

with open('position.json') as position:
    data = json.load(position)

serial = []
for x in range(0,Sensors_X*Sensors_Y*Sensors_Z):
    random = str(uuid.uuid4())
    random = random.upper()
    random = random.replace("-","") # Remove the UUID '-'
    serial.append(random[0:6])
    for line in enumerate(serial):
        if x == line[0]:
            continue
        else:
            while serial[x] == serial[line[0]]:
                random = str(uuid.uuid4())
                random = random.upper()
                random = random.replace("-","") # Remove the UUID '-'
                serial[x] = random[0:6]

# sensor locations in m
for y_val in range(-int(Sensors_Y/2),int(Sensors_Y/2)):
    for x_val in range(-int(Sensors_X/2),int(Sensors_X/2)):
        for z_val in range(-int(Sensors_Z/2),int(Sensors_Z/2)):
            indx=z_val + x_val*Sensors_Z + y_val*Sensors_X*Sensors_Z + (int(Sensors_Z/2) + int(Sensors_X/2)*Sensors_Z + int(Sensors_Y/2)*Sensors_X*Sensors_Z)
            a_dictionary = {'serial': serial[indx], 'x': x_val * Dist_btw_Sen_X + 1, 'y': y_val * Dist_btw_Sen_Y + 1, 'z': z_val * Dist_btw_Sen_Z + 1}
            data['detectors'].append(a_dictionary)

data['sensors'] = len(data['detectors'])

json_object_app = json.dumps(data, indent = 4) 
with open("position.json", "w") as outfile: 
    outfile.write(json_object_app)


In [8]:
#Serial Number Uniqueness Check

for line in enumerate(data['detectors']):
    for line2 in enumerate(data['detectors']):
        if line == line2:
            continue
        elif data['detectors'][line[0]]['serial'] == data['detectors'][line2[0]]['serial']:
            print(data['detectors'][line[0]]['serial'] + 'shoot' + data['detectors'][line2[0]]['serial'])